In [1]:
from dotenv import load_dotenv
import os
from langchain.chat_models import ChatOpenAI
from langchain.llms import OpenAI
from langchain.schema import HumanMessage
from langchain.schema import BaseOutputParser
from langchain.prompts import PromptTemplate
from langchain.prompts.chat import ChatPromptTemplate

## Configure API keys

In [4]:
load_dotenv()
openai_key = os.getenv('OPENAI_API_KEY')

## Setup chat and llm runnables

In [5]:
chatmod = ChatOpenAI(openai_api_key=openai_key)
llm = OpenAI(openai_api_key=openai_key)

## Test direct interaction with LLM and ChatModel 

In [6]:
text = "What would be a good name for a Death Metal band that loves ponies?"
messages = [HumanMessage(content=text)]

In [7]:
llm.invoke(text)

'\n\nMare of Suffering'

In [8]:
chatmod.invoke(messages)

AIMessage(content='"Dark Equine Devastation"')

## Test interaction with ChatModel via Prompt

In [9]:
prompt_text = "What would be a good name for {product}?"

prompt = PromptTemplate.from_template(prompt_text)
prompt.format(product="a Death Metal band that loves ponies")

'What would be a good name for a Death Metal band that loves ponies?'

In [10]:
system_template = "You are a friendly {topic} fan, spreading the word {topic} and its beauty."
human_template = "{text}"

chat_prompt = ChatPromptTemplate.from_messages([
    ('system', system_template),
    ('human', human_template)
])

form_chat_prompt = chat_prompt.format_messages(topic='Death Metal', text='Hi! Do you have music suggestions?')

In [11]:
chatmod.invoke(form_chat_prompt)

AIMessage(content='Absolutely! I would be more than happy to recommend some Death Metal bands and albums for you. Here are a few classics and personal favorites to get you started:\n\n1. Death - "Symbolic" (1995): This album by the legendary Chuck Schuldiner is considered a masterpiece of progressive Death Metal.\n\n2. Morbid Angel - "Altars of Madness" (1989): A pioneering band in the genre, Morbid Angel\'s debut album is a relentless assault of intense, technical riffs and brutal vocals.\n\n3. Cannibal Corpse - "Tomb of the Mutilated" (1992): Known for their graphic lyrics and aggressive sound, Cannibal Corpse is a staple in Death Metal. This album showcases their brutality and technicality.\n\n4. Obituary - "Cause of Death" (1990): Obituary\'s sophomore release is a raw and crushing Death Metal album that solidified their place in the genre.\n\n5. At the Gates - "Slaughter of the Soul" (1995): This Swedish band\'s album is a melodic Death Metal masterpiece, combining aggression with

## Test basic chatbot chain

In [12]:
class SentenceSeparatorOutputParser(BaseOutputParser):
    """Parse the output of an LLM call, by splitting it into sentences and placing each sentence in a list"""

    def parse(self, text):
        """Parse the output of the LLM call"""
        list_lines = text.strip().split('.')
        return [i.lstrip() for i in list_lines]
    
template = "You are a huge {topic} fan, spreading the word {topic} and its beauty."
human_template = "{text}"

chat_prompt = ChatPromptTemplate.from_messages([
    ("system", template),
    ("human", human_template)
])

chain = chat_prompt | ChatOpenAI() | SentenceSeparatorOutputParser()

chain.invoke({"topic":"Death Metal", "text":"What is your favourite color?"})

["As a fan of Death Metal, I don't have a specific favorite color",
 'However, many Death Metal bands incorporate dark and intense imagery, so black is often associated with the genre',
 'Black symbolizes the raw and brutal nature of Death Metal, representing the darkness and heaviness of the music',
 '']